In [25]:
import csv  # For reading and writing CSV files
import os  # For file system operations like checking paths
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For HTML parsing
import time  # For pauses between requests

In [22]:
import warnings
warnings.filterwarnings("ignore", "\nPyarrow", DeprecationWarning)
import pandas as pd
import os

def search_and_separate(url_pattern, input_file, output_file):
    # Read the CSV file
    df = pd.read_csv(input_file)

    # Filter rows containing the pattern in the 'link' column
    rows_with_pattern = df[df["link"].str.contains(url_pattern, na=False)]

    # Save the filtered rows to a new CSV file
    rows_with_pattern.to_csv(output_file, index=False)

    # Print the number of rows found
    print(
        f"Found {len(rows_with_pattern)} rows containing the pattern in the 'link' column."
    )



In [23]:
url_pattern = "https://quotidiano.repubblica.it/edicola/searchdetail\?id=http://archivio.repubblica.extra.kataweb.it/"

input_file = 'output_part1.csv'

output_file = "url_patron_1.csv"

search_and_separate(url_pattern, input_file, output_file)

Found 59519 rows containing the pattern in the 'link' column.


In [24]:
def parse(response, visited_links, visited_links_file):
  # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")



    article = soup.find("article")

    title = article.get_text(separator=" ", strip=True)

    # Extract the article link
    content = article.find("p").get_text(separator=" ", strip=True)


    date = article.find("aside").find("a").find("time").get_text(separator=" ",strip=True)

    article_data = [
         {"title": title, "content": content, "date": date, "src_url": response.url}
    ]

    # Define los nombres de los campos
    fieldnames = ["title", "content", "date", "src_url"]

    # Verifica si el archivo existe
    mode = "a" if os.path.exists("output") else "w"
        # Si el archivo existe, abre en modo "append"
    with open("output", "a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Escribe los datos del artículo en el archivo
        writer.writerows(article_data)


    # else:
    #     # Si el archivo no existe, abre en modo de escritura normal
    #     with open("output", "w", newline="", encoding="utf-8") as file:
    #         writer = csv.DictWriter(file, fieldnames=fieldnames)

    #         # Escribe los datos del artículo en el archivo
    #         writer.writerows(article_data)



In [ ]:
import csv  # For reading and writing CSV files
import os  # For file system operations like checking paths
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For HTML parsing
import time  # For pauses between requests

def scraper(visited_links_file, start_links_csv):
    # Comprobar si el archivo de URL visitadas existe para ver si es la primera ejecución o una continuación después de un error
    if os.path.exists(visited_links_file):
       # Abra el archivo y cree un conjunto de URL visitadas
        with open(visited_links_file) as f:
            reader = csv.reader(f)
            visited_links = {row[0] for row in reader if row}
    else:
        # If the file doesn't exist, it's the first run
        visited_links = set()

    # Open the start URL CSV and iterate through it
    with open(f"./{start_links_csv}", "r", newline="") as file:
        reader = csv.DictReader(file)

        iteration = 1

        for row in reader:
            link = row["link"]

            pattern = "https://quotidiano.repubblica.it/edicola/searchdetail\?id=http://archivio.repubblica.extra.kataweb.it/"

            if pattern in link:

                print(f"iteration number {iteration}")

                # Check the link against the already visited ones
                if link not in visited_links:
                    visited_links.add(link)

                    # Make the HTTP request
                    response = requests.get(link)
                    if response.status_code == 200:
                        # Call the parse method to handle the response
                        parse(response, visited_links, visited_links_file)
                    elif response.status_code == 403:

                        print("status is 403\n" * 3)

                        print("start the 4 minute wait")

                        # Wait for 4 minutes before retrying
                        for i in range(1, 5):
                            time.sleep(60)
                            print(f"{i} minutes have passed")
                iteration += 1

            print(
                f"Check the result: xxxxx"
            )
            print("************** End **************")

In [ ]:


    # Method to handle parsing of the response
    def parse(self, response):
        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all article elements on the page
        articles = soup.find_all("article")

        # List to store extracted article data
        article_data = []

        for article in articles:
            # Get the inner HTML of the title anchor tag
            anchor_html = article.find("h1").find("a")

            # Extract clean text from the title
            title = anchor_html.get_text(separator=" ", strip=True)

            # Extract the article link
            link = anchor_html["href"]

            # Extract the publication date
            aside_element = article.find("aside").find_all("a")
            date = aside_element[-1].get_text(separator=" ", strip=True)

            # Add article data to the list
            article_data.append(
                {"title": title, "link": link, "date": date, "page_url": response.url}
            )

        # Write the extracted article data to the CSV file
        with open(
            self._build_visited_urls_file(use_output_suffix=True),
            "a",
            newline="",
            encoding="utf-8",
        ) as file:
            writer = csv.DictWriter(
                file, fieldnames=["title", "link", "date", "page_url"]
            )

            # Write the header if the file is empty
            if (
                os.stat(self._build_visited_urls_file(use_output_suffix=True)).st_size
                == 0
            ):
                writer.writeheader()

            # Write a row for each article data
            for data in article_data:
                writer.writerow(data)

        # Add the URL to the visited file
        with open(self.visited_urls_file, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([response.url])

    def _build_visited_urls_file(self, use_output_suffix=False):

        # Get the base filename of the start URLs CSV file without the extension
        base_filename = os.path.splitext(os.path.basename(self.start_urls_csv))[0]

        # Remove the date part from the base filename
        base_filename_without_date = base_filename.split("_")[0]

        # Construct the filename suffix based on the use_output_suffix parameter
        if use_output_suffix:
            filename_suffix = "_output.csv"
        else:
            filename_suffix = "_visited_urls.csv"

        # Construct the filename for the visited URLs file
        visited_urls_filename = f"{base_filename_without_date}{filename_suffix}"

        return f"./output/{visited_urls_filename}"

In [ ]:
"""

https://quotidiano.repubblica.it/edicola/searchdetail?id=http://archivio.repubblica.extra.kataweb.it/archivio/repubblica
/2019/11/02/incosciente-non-fu-stupro-pena-ridotta-per-il-protesta18.html&hl=&query=sopa&field=nel+testo&testata=repubblica&newspaper=REP&edition=nazionale&zona=sfoglio&ref=search



https://ricerca.repubblica.it/repubblica/archivio/repubblica
https://ricerca.repubblica.it/repubblica/archivio/repubblica/2006/08/sezioni/cronaca/milano-ladri-bambini/milano-ladri-bambini/milano-ladri-bambini.html?ref=search



https://ricerca.repubblica.it/repubblica/archivio/repubblica/2019/11/02/incosciente-non-fu-stupro-pena-ridotta-per-il-protesta18.html&hl=&query=sopa&field=nel+testo&testata=repubblica&newspaper=REP&edition=nazionale&zona=sfoglio&ref=search


http://www.repubblica.it/sport/vari/2015/06/25/news/giochi_europei_quattro_azzurri_in_finale_per_la_boxe-117672858/?ref=search


http://www.repubblica.it/2006/08/sezioni/cronaca/milano-ladri-bambini/mila   no-ladri-bambini/milano-ladri-bambini.html?ref=search


"""

In [1]:
import csv  # For reading and writing CSV files
import os  # For file system operations like checking paths
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For HTML parsing
import time  # For pauses between requests